## Cassandra Python Assignment - Saranya Chintalapati


### Import packages

In [2]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [3]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [4]:
session.execute("DROP KEYSPACE IF EXISTS bd23")

In [5]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [7]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Create a Inventory table

In [8]:
session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inventory ( 
    SKU TEXT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(SKU)
);
""")

### Insert data to Inventory table


In [9]:
# CQL INSERT statement
insert_query = """
INSERT INTO bd23.inventory(SKU, name, description, warehouse_num)
VALUES (?, ?, ?, ?)
"""

prepared_insert = session.prepare(insert_query)

# Define the data to insert as a list of tuples
data_to_insert = [
    ('SKU1001', 'Air Conditioner', 'Keep your home cool and comfortable with our energy-efficient air conditioner.', 123),
    ('ADS1002', 'Toaster', 'Blend, mix, and puree with ease using our versatile blender.', 456),
    ('BGS1003', 'Dishwasher', 'Experience the convenience of our modern dishwasher.', 123),
    ('KJH1004', 'Microwave', 'Power up your kitchen with our high-performance microwave oven.', 789),
    ('HGD1005', 'Kettle', 'Efficiency meets style in our sleek electric kettle.', 987),
    ('SER1006', 'Washing Machine F', 'Upgrade your laundry game with our top-notch washing machine.', 459)
]
# Insert each row in a loop
for row in data_to_insert:
    session.execute(prepared_insert, row)

In [10]:
rows = session.execute("select (SKU, name, description, warehouse_num) from bd23.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


SKU=HGD1005, name=Kettle, description=Efficiency meets style in our sleek electric kettle., warehouse_num=987
SKU=ADS1002, name=Toaster, description=Blend, mix, and puree with ease using our versatile blender., warehouse_num=456
SKU=SER1006, name=Washing Machine F, description=Upgrade your laundry game with our top-notch washing machine., warehouse_num=459
SKU=BGS1003, name=Dishwasher, description=Experience the convenience of our modern dishwasher., warehouse_num=123
SKU=SKU1001, name=Air Conditioner, description=Keep your home cool and comfortable with our energy-efficient air conditioner., warehouse_num=123
SKU=KJH1004, name=Microwave, description=Power up your kitchen with our high-performance microwave oven., warehouse_num=789


### Select query to seach all products with warehouse_num = 123

In [12]:
rows = session.execute("select (SKU, name, description, warehouse_num) from bd23.inventory where warehouse_num = 123 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

BGS1003, Dishwasher, Experience the convenience of our modern dishwasher., 123
SKU1001, Air Conditioner, Keep your home cool and comfortable with our energy-efficient air conditioner., 123
